# SoH estimatino comparaison with real world SoH readout
On the 14th of January, we recieved a small sample of SoH readouts from a few tesla vehicles.  
This notebook serves as a benchmark to compare our SoH estimation with the real world SoH readout.  

## Setup

In [ ]:
import plotly.express as px

from core.plt_utils import *
from core.sql_utils import *
from core.stats_utils import *
from core.pandas_utils import *
from transform.fleet_info.main import fleet_info
from transform.raw_results.tesla_results import get_results
from transform.raw_results.get_tesla_soh_readouts import get_aviloo_soh_readouts
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries

## Data extraction

In [ ]:
charges = get_results()
charges

In [ ]:
raw_soh_per_vin = (
    charges
    .groupby("vin", observed=True, as_index=False)
    .agg(
        soh=("soh", "median"),
        odometer=("odometer", "last"),
        date=("date", "last"),
    )
    .merge(fleet_info, "left", "vin")
)
sanity_check(raw_soh_per_vin)

In [ ]:
ground_truth = get_aviloo_soh_readouts()
ground_truth

## Comparison

In [ ]:
raw_soh_per_vin_soh_per_vin_with_ground_truth = (
    raw_soh_per_vin
    .merge(ground_truth[["vin", "soh_readout"]], "inner", "vin")
    .eval("residual_soh = soh - soh_readout")
    .assign(abs_residual_soh=lambda x: x["residual_soh"].abs())
)
display(raw_soh_per_vin_soh_per_vin_with_ground_truth)
display(raw_soh_per_vin_soh_per_vin_with_ground_truth.describe())

## Conclusion
We can see that the SoH estimation is quite good, with a mean residual of 0.016 (1.6%) and a standard deviation of 0.013 (1.3%).  